In [ ]:
import os
import torch
import torch.nn as nn
from utility import train_test_split
from numpy import unique
from utility import load_data, batch_training_generator
from torch_utility import accuracy, fully_connected, Flatten
from sequential import Sequential

In [ ]:
image_dir = os.getcwd() + '/grey_images_32' 
X_raw, labels = load_data(image_dir)

In [ ]:
length, width = X_raw[0].shape
X = torch.Tensor(X_raw).type(torch.float32)
X = X.view(-1, 1, length, width)
X /= 255
unique_y = unique(labels)
y_dim = len(unique_y)
num_unique_labels = len(unique_y)
mapping = {label: i for i, label in enumerate(unique_y)}
numeric_labels = torch.Tensor([mapping[label] for label in labels]).type(torch.long)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, numeric_labels, test_size=0.1,
                                                    shuffle=False)

In [ ]:
N = len(X_train)

# 32 X 32 X 1
layers = [nn.Conv2d(1, 32, 5), # 28 X 28 X 32
          nn.ReLU(),
          nn.MaxPool2d(2, 2), # 14 X 14 X 32
          nn.Conv2d(32, 32, 5), # 10 X 10 X 32
          nn.ReLU(),
          nn.MaxPool2d(2, 2), # 5 X 5 X 32
          Flatten(),
          nn.Linear(5 * 5 * 32, y_dim)
         ]
model = Sequential(*layers)
model

In [ ]:
epochs = 10
batch_size = 32
op = torch.optim.Adam(model.parameters(), lr=.001)
loss_fn = nn.CrossEntropyLoss(reduction='sum')
for epoch_num in range(epochs):
    epoch_loss = 0
    training = batch_training_generator(X_train, y_train, batch_size,
                                        shuffle=True)
    for x_set, y_set in training:
        op.zero_grad()
        output = model(x_set)
        loss = loss_fn(output, y_set)
        loss /= N
        epoch_loss += float(loss)
        loss.backward()
        op.step()
    print(epoch_num, epoch_loss, accuracy(model, X_train, y_train))
model.append(nn.Softmax(dim=1))
print(accuracy(model, X_test, y_test))